## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [113]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo if len(codigo) == 4 else None
        self.nombre = nombre if 1 <= len(nombre) <= 100 else None
        self._precio = precio if 10 <= precio <= 10000 else None
        self.tipo = tipo if len(tipo) <= 20 else None
        self._cantidad = cantidad if 0 <= cantidad <= 1000 else None

    def valorTotal(self):
        return self._precio * self._cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if 10 <= valor <= 10000:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if 0 <= valor <= 1000:
            self._cantidad = valor

    def __repr__(self):
        return f"{self.codigo}: {self.nombre} ({self.tipo}) - ${self.precio} x {self.cantidad}"


In [114]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [111]:
class Oferta:
    def __init__(self, descripcion="Sin descripción", codigos=None, tipos=None):
        self.descripcion = descripcion if 1 <= len(descripcion) <= 100 else "Sin descripción"
        self.codigos = codigos if codigos else []
        self.tipos = tipos if tipos else []

    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad * self.descuento) / 100
        return 0

class Oferta2x1(Oferta):
    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            productos_gratis = cantidad // 2
            return producto.precio * productos_gratis
        return 0


In [112]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [122]:
import pandas as pd


class Catalogo:
    def __init__(self, archivo_csv=None):
        self.ofertas = []
        if archivo_csv:
            self.productos_df = pd.read_csv(archivo_csv)
            self.productos = self._cargar_productos()
        else:
            self.productos = []

    def _cargar_productos(self):
       productos = []
       for _, row in self.productos_df.iterrows():
        codigo = str(row['codigo']).zfill(4)  # Asegura que el código sea una cadena de 4 dígitos
        producto = Producto(codigo, row['nombre'], row['precio'], row['tipo'], row['cantidad'])
        productos.append(producto)
       return productos



    @staticmethod
    def leer(archivo_csv):
        return Catalogo(archivo_csv)

    def guardar(self, archivo_csv):
        df = pd.DataFrame([{
            'codigo': p.codigo,
            'nombre': p.nombre,
            'precio': p.precio,
            'tipo': p.tipo,
            'cantidad': p.cantidad
        } for p in self.productos])
        df.to_csv(archivo_csv, index=False)


    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta.calcularDescuento(producto, cantidad)
        return 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos)
    
    def informe(self):
        tipos = set(p.tipo for p in self.productos)
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0
        informe = (
            f"INFORME CATALOGO\n"
            f"Cantidad de productos: {self.cantidadProductos}\n"
            f"Cantidad de unidades: {self.cantidadUnidades}\n"
            f"Precio Promedio: ${precio_promedio:.2f}\n"
            f"Valor total: ${self.valorTotal:.2f}\n"
            f"Tipos de productos:\n"
        )
        for tipo in tipos:
            unidades = sum(p.cantidad for p in self.productos if p.tipo == tipo)
            precio_prom = sum(p.precio * p.cantidad for p in self.productos if p.tipo == tipo) / unidades if unidades > 0 else 0
            informe += f"  - {tipo}: {unidades}u x ${precio_prom:.2f}\n"
        informe += "Ofertas:\n"
        for oferta in self.ofertas:
            informe += f"  - {oferta.descripcion}\n"
        return informe

    
    
        
    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
        else:
            print(f"No hay suficiente stock de {producto.nombre}.")


In [130]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(descripcion="Oferta 2x1 en galletas", tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, descripcion="Descuento del 10% en Coca Cola y Sonrisa", codigos=['0001', '0003']))


assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: 4" in informe
assert "Cantidad de unidades: 97" in informe
assert "Precio Promedio: $" in informe
assert "Valor total: $" in informe
assert "Tipos de productos:" in informe
assert "galleta" in informe
assert "gaseosa" in informe
assert "Ofertas:" in informe
assert "Oferta 2x1 en galletas" in informe
assert "Descuento del 10% en Coca Cola y Sonrisa" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [125]:
class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = nombre if 1 <= len(nombre) <= 100 else None
        if self._nombre is None:
            raise ValueError("El nombre debe tener entre 1 y 100 caracteres.")
        self._cuit = cuit if self.validar_cuit(cuit) else None
        if self._cuit is None:
            raise ValueError("CUIT inválido.")

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, nuevo_nombre):
        if 1 <= len(nuevo_nombre) <= 100:
            self._nombre = nuevo_nombre

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, nuevo_cuit):
        if self.validar_cuit(nuevo_cuit):
            self._cuit = nuevo_cuit

    def validar_cuit(self, cuit):
        partes = cuit.split('-')
        return len(partes) == 3 and len(partes[0]) == 2 and len(partes[1]) == 8 and len(partes[2]) == 1 and all(p.isdigit() for p in partes)


In [126]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [127]:
from datetime import datetime

class Factura:
    contador_facturas = 0  

    @staticmethod
    def ultimaFactura(numero):
        Factura.contador_facturas = numero

    @staticmethod
    def nuevoNumero():
        Factura.contador_facturas += 1
        return Factura.contador_facturas

    def __init__(self, catalogo, cliente):
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now()
        self.cliente = cliente
        self.productos = []  
        self.subtotal = 0
        self.descuentos = 0
        self.total = 0
        self.catalogo = catalogo

    def agregar(self, producto, cantidad):
        if producto is None:
            raise ValueError("El producto no puede ser None.")
        
        # Verifica si hay suficiente cantidad en el catálogo para vender
        if producto.cantidad < cantidad:
            raise ValueError(f"No hay suficiente stock de {producto.nombre} para vender {cantidad} unidades.")
        
        # Resta la cantidad vendida del stock disponible
        producto.cantidad -= cantidad
        
        # Verificar si el producto ya está en la lista de la factura
        for i, (p, cant, sub, desc) in enumerate(self.productos):
            if p.codigo == producto.codigo:
                # Si el producto ya está, acumular la cantidad y recalcular el subtotal y el descuento
                nueva_cantidad = cant + cantidad
                nuevo_subtotal = producto.precio * nueva_cantidad
                nuevo_descuento = self.catalogo.calcularDescuento(producto, nueva_cantidad)
                self.subtotal += (nuevo_subtotal - sub)
                self.descuentos += (nuevo_descuento - desc)
                self.total = self.subtotal - self.descuentos
                self.productos[i] = (producto, nueva_cantidad, nuevo_subtotal, nuevo_descuento)
                return
        
        # Si el producto no está, agregarlo como nuevo
        subtotal = producto.precio * cantidad
        descuento = self.catalogo.calcularDescuento(producto, cantidad)
        self.subtotal += subtotal
        self.descuentos += descuento
        self.total = self.subtotal - self.descuentos
        self.productos.append((producto, cantidad, subtotal, descuento))




    @property
    def cantidadProductos(self):
        return len(set(p[0] for p in self.productos))

    @property
    def cantidadUnidades(self):
        return sum(p[1] for p in self.productos)
    
    def imprimir(self):
        output = f"Factura Nº: {self.numero}\n"
        output += f"Fecha: {self.fecha}\n"
        output += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        for producto, cantidad, subtotal, descuento in self.productos:
            output += f"{cantidad} {producto.nombre:<20}x ${producto.precio} = ${subtotal:.2f}\n"
            # Mostrar el descuento con la descripción de la oferta si aplica
            if descuento > 0:
                for oferta in self.catalogo.ofertas:
                    if oferta.esAplicable(producto, cantidad):
                        if isinstance(oferta, OfertaDescuento):
                            output += f"      Descuento {oferta.descuento}%: -${descuento:.2f}\n"
                        elif isinstance(oferta, Oferta2x1):
                            output += f"      Oferta 2x1: -${descuento:.2f}\n"
        output += f"\nSubtotal:   ${self.subtotal:.2f}\n"
        output += f"Descuentos: ${self.descuentos:.2f}\n"
        output += f"TOTAL:      ${self.total:.2f}\n"
        return output



        
       


In [128]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [131]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)


assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
